# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = '../WeatherPy/output_data_file'
df = pd.read_csv(file, index_col = 0)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.60,73.08,84.92,72,8,9.95,MV,1587410828
1,Kifrī,34.70,44.97,74.28,33,2,5.50,IQ,1587410829
2,Busselton,-33.65,115.33,62.01,61,19,10.83,AU,1587410829
3,Qaanaaq,77.48,-69.36,12.51,85,97,10.22,GL,1587410829
4,Thompson,55.74,-97.86,21.20,67,90,12.75,CA,1587410829
...,...,...,...,...,...,...,...,...,...
579,Yenotayevka,47.25,47.03,56.08,68,100,11.07,RU,1587410965
580,Melur,10.05,78.33,83.68,60,30,4.83,IN,1587410965
581,Khuzhir,53.19,107.34,21.76,86,3,4.56,RU,1587410965
582,Karasburg,-28.02,18.75,72.27,22,44,11.79,NaN,1587410965


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = df[['Lat','Lng']]
humidity = df['Humidity'].astype(float)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_df = pd.DataFrame(df, columns = ['City','Country','Max Temp','Wind Speed','Cloudiness','Lat','Lng'])
max_temp = (new_df['Max Temp'] <= 80) & (new_df['Max Temp'] > 70)
wind_speed = (new_df['Wind Speed'] < 10)
cloudiness = (new_df['Cloudiness'] == 0)

new_df = new_df[max_temp & wind_speed & cloudiness]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(new_df, columns = ['City','Country','Lat','Lng'])
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Arraial do Cabo,BR,-22.97,-42.03,
93,Sittwe,MM,20.15,92.90,
133,Chui,UY,-33.70,-53.46,
136,Cidreira,BR,-30.18,-50.21,
141,Castro,BR,-24.79,-50.01,
212,Balkanabat,TM,39.51,54.37,
267,Pyn U Lwin,MM,22.03,96.47,
340,Kahului,US,20.89,-156.47,
384,Santa Vitória do Palmar,BR,-33.52,-53.37,
510,Rocha,UY,-34.48,-54.33,


In [8]:
hotel_name = []

params = {
    'radius': 5000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params).json()
    hotel_name.append(response['results'][0]['name'])


In [9]:
hotel_df['Hotel Name'] = hotel_name

In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Arraial do Cabo,BR,-22.97,-42.03,Hotel da Canoa
93,Sittwe,MM,20.15,92.90,Yuzana Aung Hotel
133,Chui,UY,-33.70,-53.46,Bertelli Chuí Hotel
136,Cidreira,BR,-30.18,-50.21,Pond Country Club
141,Castro,BR,-24.79,-50.01,Central Palace Hotel - Castro (PR)
212,Balkanabat,TM,39.51,54.37,Nebitchi Hotel
267,Pyn U Lwin,MM,22.03,96.47,Aureum palace Hotel
340,Kahului,US,20.89,-156.47,Maui Beach Hotel
384,Santa Vitória do Palmar,BR,-33.52,-53.37,Mirante Hotel
510,Rocha,UY,-34.48,-54.33,Beleza Pura B&B


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))